# Progetto Fondamenti di Machine Learning

Il progetto si articola in **quattro fasi principali**: la definizione manuale di classificatori sul dataset "manuale.csv", l'analisi esplorativa del dataset "training.csv" per verificare la qualità dei dati e identificare pattern significativi, l'ottimizzazione delle performance dei classificatori implementati, e infine l'addestramento di modelli avanzati utilizzando Scikit-Learn con separazione appropriata tra training set e test set.


---
# Primo task

Nel primo task ci viene chiesto di definire manualmente più classificatori sul file "manuale.csv", adottando almeno due dei modelli illustrati a lezione, e valutando le prestazioni da loro ottenute sullo stesso file “manuale.csv”; dopo aver illustrato i passi per adattare i modelli ai dati, implementare i classificatori in Python, utilizzando eventualmente delle API;

### Import necessari

In questa sezione verranno inseriti i vari import necessari per eseguire il primo task.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression # per costruire il modello di regressione logistica
from sklearn.metrics import accuracy_score # utile per le metriche di accuratezza
from sklearn.tree import DecisionTreeClassifier # per costruire il modello di albero decisionale 
from sklearn.preprocessing import StandardScaler # per scalare i dati (utile per il modello di regressione logistica)

### Esaminiamo la struttura del dataset

Per prima cosa vogliamo esaminare la struttura del dataset usando più classificatori, in questo modo ci permetterà nella preparazione e comprensione dei dati:

In [5]:
# Carichiamo il file CSV specificando il delimitatore, nel nostro caso è il punto e virgola.
df = pd.read_csv("../specifiche/data/manuale.csv", sep=';')

# Esaminiamo la struttura del dataset

# Per prima cosa ci ricaviamo la dimensione del dataset
print("Dimensioni del dataset:", df.shape)

Dimensioni del dataset: (25, 12)


In [6]:
# Dopodiché visualizzzaimo tutto il dataset essendo abbastanza piccolo
df

,Età,Genere,Fumatore,Ex-Fumatore,Non-Fumatore,Data,Frequenza Cardiaca,SpO2,OssigenoTerapia,DopoTest6Minuti,Temperatura,Allerta?
0,43,F,False,False,True,05/06/2024,91,99,venturi_mask,False,"36,3",GREEN
1,79,M,False,True,False,10/08/2024,70,93,venturi_mask,False,36,YELLOW
2,81,M,True,False,False,04/08/2024,124,91,no,False,"36,5",YELLOW
3,81,M,False,False,True,16/08/2024,120,92,ox,False,"36,6",RED
4,48,F,False,False,True,27/06/2024,97,95,venturi_mask,False,"36,1",YELLOW
5,82,M,False,False,True,02/02/2024,121,92,ox,False,"36,6",RED
6,72,M,False,False,True,28/06/2024,68,98,no,False,36,GREEN
7,78,M,False,True,False,16/07/2024,70,95,no,False,"36,3",GREEN
8,82,M,False,False,True,24/03/2024,123,90,ox,False,"36,5",RED
9,81,M,False,False,True,24/05/2024,126,92,no,False,"36,5",YELLOW


Da questo output possiamo trarre che il dataset rappresenta dati medici di pazienti per la classificazione del livello di allerta (GREEN/YELLOW/RED). Essendo un dataset con dimensioni ridotte verifichiamo se è presente una distribuzione bilanciata delle classi target. 

In [4]:
# Facciamo il conteggio  
target_counts = df['Allerta?'].value_counts()
print(target_counts)
print(f"\nGREEN({target_counts['GREEN']}), YELLOW({target_counts['YELLOW']}), RED({target_counts['RED']})")

Allerta?
YELLOW    9
GREEN     8
RED       8
Name: count, dtype: int64

GREEN(8), YELLOW(9), RED(8)


Da questo output possiamo confermare che è presente una **distribuzione ben bilanciata**. Passiamo ad analizzare i tipi di dato delle varie colonne.

In [5]:
print(df.dtypes)

Età                    int64
Genere                object
Fumatore                bool
Ex-Fumatore             bool
Non-Fumatore            bool
Data                  object
Frequenza Cardiaca     int64
SpO2                   int64
OssigenoTerapia       object
DopoTest6Minuti         bool
Temperatura           object
Allerta?              object
dtype: object


## Osservazioni a seguito dell'analisi del dataset

Durante l'analisi sono emmersi **quattro problemi** che richiedono interventi di pulizia. Inoltre, per garantire la compatibilità con gli algoritmi di machine learning, le variabili dovranno essere convertite in un formato numerico.

### Problema 1

Le temperature sono registrate con la **virgola come separatore decimale** invece del punto, seguendo la convenzione italiana. Questo impedisce la conversione automatica in formato numerico. Senza conversione, la colonna risulterebbe inutilizzabile.

Per risolvere questo problema passiamo a sostituire la virgola con il punto nella colonna temperatura. 

In [6]:
# Carichiamo il dataset in un'altra variabile che rappresenterà il nostro dataset 'pulito' 

df_clean = pd.read_csv("manuale.csv", sep=';')
df_clean['Temperatura'] = df_clean['Temperatura'].str.replace(',', '.').astype(float)

# Per verificare che la modifica sia andata a buon fine stampiamo le prime 5 righe del dataset

df_clean.head()

,Età,Genere,Fumatore,Ex-Fumatore,Non-Fumatore,Data,Frequenza Cardiaca,SpO2,OssigenoTerapia,DopoTest6Minuti,Temperatura,Allerta?
0,43,F,False,False,True,05/06/2024,91,99,venturi_mask,False,36.3,GREEN
1,79,M,False,True,False,10/08/2024,70,93,venturi_mask,False,36.0,YELLOW
2,81,M,True,False,False,04/08/2024,124,91,no,False,36.5,YELLOW
3,81,M,False,False,True,16/08/2024,120,92,ox,False,36.6,RED
4,48,F,False,False,True,27/06/2024,97,95,venturi_mask,False,36.1,YELLOW


## Problema 2

Lo status del fumatore è rappresentato tramite **3 colonne booleane separate**:
- `Fumatore`: true/false
- `Ex-Fumatore`: true/false  
- `Non-Fumatore`: true/false

**Analisi della ridondanza:**
- Ogni paziente ha **esattamente una** delle tre colonne a `true`
- Esse infatti rappresentano la stessa informazione in formato scomodo

**Esempio:**
```
Fumatore: false | Ex-Fumatore: true  | Non-Fumatore: false  → Ex-Fumatore
Fumatore: true  | Ex-Fumatore: false | Non-Fumatore: false  → Fumatore
Fumatore: false | Ex-Fumatore: false | Non-Fumatore: true   → Non-Fumatore
```

Per risolvere questo problema creiamo una nuova colonna chiamatas `StatoFumo` che indicherà i vari status indicati nelle altre colonne.

In [7]:
# Per prima cosa definiamo le condizioni per categoriizzare lo status da fumatore
conditions = [
    # Prima condizione: fumatore
    df_clean['Fumatore'] == 1,
    # Seconda condizione: ex fumatore
    (df_clean['Fumatore'] == 0) & (df_clean['Ex-Fumatore'] == 1)
]

# Definiamo le scelte per ogni condizione
choices = ['Fumatore', 'Ex-Fumatore']

# Creiamo una nuova colonna usando numpy.select con 'Non-Fumatore' come status di default
df_clean['StatoFumo'] = np.select(conditions, choices, default='Non-Fumatore')

# Eliminiamo le colonne che non ci servono più
df_clean = df_clean.drop(columns=['Fumatore', 'Ex-Fumatore', 'Non-Fumatore'])

# Stampiamo le prime 5 righe per verificare che tutto sia corretto.
df_clean.head()

,Età,Genere,Data,Frequenza Cardiaca,SpO2,OssigenoTerapia,DopoTest6Minuti,Temperatura,Allerta?,StatoFumo
0,43,F,05/06/2024,91,99,venturi_mask,False,36.3,GREEN,Non-Fumatore
1,79,M,10/08/2024,70,93,venturi_mask,False,36.0,YELLOW,Ex-Fumatore
2,81,M,04/08/2024,124,91,no,False,36.5,YELLOW,Fumatore
3,81,M,16/08/2024,120,92,ox,False,36.6,RED,Non-Fumatore
4,48,F,27/06/2024,97,95,venturi_mask,False,36.1,YELLOW,Non-Fumatore


## Problema 3

Nel contesto di un **sistema di triage medico**, la data della visita **non dovrebbe influenzare** il livello di allerta del paziente. I parametri vitali (frequenza cardiaca, SpO2, temperatura) sono **istantanei** e indipendenti dal momento della misurazione.

Potrebbero esistere dei pattern temporali, però, la decisione è stata quella di rimuovere la colonna della data per mantere il focus sui **parametri clinici diretti.**

In [8]:
# Rimozione della colonna Data
df_clean = df_clean.drop(columns=['Data'])

# Stampiamo le colonne presenti per verificare 
print(f"• Colonne presenti: {list(df_clean.columns)}")

• Colonne presenti: ['Età', 'Genere', 'Frequenza Cardiaca', 'SpO2', 'OssigenoTerapia', 'DopoTest6Minuti', 'Temperatura', 'Allerta?', 'StatoFumo']


## Problema 4

Gli algoritmi di machine learning tradizionali operano in spazi vettoriali numerici e non possono processare direttamente variabili categoriche. Per questo motivo, le variabili qualitative verranno trasformate in **valori interi** attraverso una mappatura manuale.

Ad esempio:
- `M` o `F` per la colonna di Genere, verrano convertiti in `0` oppure `1`.
- Per `StatoFumo`, verrà definita una scala ordinata con
  - Non-Fumatore = 0
  - Ex-Fumatore = 1
  - Fumatore = 2

In [9]:
# Per prima cosa creiamo i vari mapping originali

# Mappatura originale per la colonna Genere
genere_map = {'M': 0, 'F': 1}
# Mappatura originale per la colonna OssigenoTerapia
ossigeno_map = {'no': 0, 'venturi_mask': 1, 'ox': 2}
# Mappatura originale per la colonna allerta
allerta_map = {'GREEN': 0, 'RED': 1, 'YELLOW': 2}
# Mappatura originale per la colonna StatoFumo
stato_map = {'Non-Fumatore': 0, 'Ex-Fumatore': 1, 'Fumatore': 2}

# Conversione numerica della colonna genere
df_clean['Genere'] = df_clean['Genere'].map(genere_map)
# Conversione numerica della colonna OssigenoTerapia
df_clean['OssigenoTerapia'] = df_clean['OssigenoTerapia'].map(ossigeno_map)
# Conversione numerica della colonna Allerta?
df_clean['Allerta?'] = df_clean['Allerta?'].map(allerta_map)
# Conversione numerica della colonna StatoFumo
df_clean['StatoFumo'] = df_clean['StatoFumo'].map(stato_map)

# Alla fine della conversione, eliminiamo le righe che hanno valori nulli
df_clean = df_clean.dropna()


## Implementazione dei classificatori

Per questo progetto sono stati selezionati due algoritmi di machine learning complementari:
- Logistic regression, che nel nostro caso ci è molto utile per **classificaizone multi-classe**.
- Decision tree, in quanto la struttura ad albero rispecchia il **ragionamento medico decisionale** e può catturare relazione cmplesse che potrebbero essere non lineari.

Per quanto riguarda il modello di **regressione logistica**, in quanto sensibile alla scala delle variabili, ossia features con range molto diversi (es. l'età:20-80 e frequenza cardiaca: 60-200), è necessaria la **standardizzazione delle features** in questo modo i dati vengono trasformati per far si che tutte le features contribuiscono equamente al modello.

Per prima cosa definiamo il nostro target, le nostre features sia standardizzate che non (in quanto il modello di decision tree non ha bisogno della standardizzazione delle features).

In [31]:
# Target: variabile da predire
y = df_clean['Allerta?']

# Features: tutte le colonne tranne il target
X = df_clean.drop(['Allerta?'], axis=1)

# Standardizzazione per Logistic Regression
scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X)

print(f"Shape delle features: {X.shape}")
print(f"Shape del target: {y.shape}")
print(f"Shape delle features standardizzate: {X_scaled.shape}")

Shape delle features: (25, 8)
Shape del target: (25,)
Shape delle features standardizzate: (25, 8)


### Logistic Regression

Il primo classificatore implementato è la regressione logistica, che richiede l'utilizzo delle features standardizzate. Una volta addestrato il modello sui dati di training, vengono generate le predizioni sui livelli di allerta e **calcolata l'accuratezza** per misurare l'efficacia del classificatore nel distinguere correttamente tra le diverse classi.

In [11]:
# Modello 1: Logistic Regression

# Creiamo un'istanza per il modello
model_lr = LogisticRegression()

# Dopodiché addestriamo il modello sulle features e sulla variabile target
model_lr.fit(X_scaled, y)

LogisticRegression()

In [12]:
# Predizione e valutazione del modello

# Genera le predizioni utilizzando il modello addestrato
y_pred_lr = model_lr.predict(X_scaled)

# Calcola l'accuratezza confrontando predizioni e valori reali
accuracy_lr = accuracy_score(y, y_pred_lr)

# Stampa l'accuratezza in percentuale
print("Accuratezza percentuale del modello Logistic Regression:", accuracy_lr * 100) # Print the accuracy as a percentage

 Percentage accuracy of the model 92.0


Il modello di regressione logistica raggiunge **un'accuratezza del 92%**, dimostrando un'elevata capacità predittiva nel classificare i livelli di allerta

### Decision Tree Classifier
Il secondo classificatore implementato è l'albero decisionale, un algoritmo non lineare che costruisce un modello predittivo attraverso una struttura gerarchica ad albero, grazie alla sua struttura che **rispecchia il processo decisionale umano**. A differenza della regressione logistica, questo modello non richiede la standardizzazione delle features.

In [29]:
# Modello 2: Decision Tree

# Creiamo un'istanza per il modello
model_dt = DecisionTreeClassifier()

# Addestra il modello sulle features non standardizzate e sulla variabile target
model_dt.fit(X, y)

DecisionTreeClassifier()

In [30]:
# Genera le predizioni utilizzando l'albero decisionale addestrato
y_pred_dt = model_dt.predict(X)

# Calcola l'accuratezza del modello Decision Tree
accuracy_dt = accuracy_score(y, y_pred_dt)


print("Accuratezza percentuale del modello Decision Tree:", accuracy_dt * 100) # Print the accuracy as a percentage

 Percentage accuracy of the model 92.0


Il Decision Tree raggiunge **un'accuratezza del 92%**, equiparando le performance della regressione logistica e confermando l'efficacia di entrambi gli approcci.

---

# Secondo task

Nel secondo task ci viene chiesto di verificare che il dataset “training.csv” non contenga osservazioni palesemente errate ed effettuare l’analisi esplorativa del dataset rappresentando i risultati anche in forma grafica (boxplot e/o pairplot e matrice di correlazione).